In [1]:
from datetime import datetime, timezone, timedelta

date=datetime.now(timezone(timedelta(hours=9))).isoformat().split("T")[0]

In [2]:
date="2025-11-11"

In [3]:
import time
import random
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

def custom_request(url):
    # 브라우저 유사 헤더
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/123.0.0.0 Safari/537.36"
        ),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Encoding": "gzip, deflate, br",
        # 목록 페이지 → 상세 페이지 흐름을 재현하려면 Referer를 실제 이전 페이지로 설정
        "Referer": url,
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
    }

    # 세션 + 재시도(403/429/5xx 백오프)
    retry = Retry(
        total=5,
        backoff_factor=1,  # 1,2,4,8,...
        status_forcelist=[403, 429, 500, 502, 503, 504],
        allowed_methods=["GET", "HEAD", "OPTIONS"]
    )
    adapter = HTTPAdapter(max_retries=retry)

    with requests.Session() as s:
        s.headers.update(headers)
        s.mount("https://", adapter)
        s.mount("http://", adapter)

        # 너무 빠른 요청을 피하기 위한 랜덤 지연
        time.sleep(random.uniform(0.8, 1.6))

        res = s.get(url, timeout=15)
        if res.status_code!=200:
            print(f"crawling failed from url : {url}!")
            return
    
    soup=BeautifulSoup(res.text, 'html.parser')
    return soup

In [4]:
def get_last_page(date):
    processed_date=date.replace("-","")
    baseUrl=f"https://coinreaders.com/search.html?submit=submit&search=%EC%9D%B4%EB%8D%94%EB%A6%AC%EC%9B%80&search_exec=all&news_order=1&search_section=all&search_and=1&search_start_day={processed_date}&search_end_day={processed_date}&page="

    page=1

    url = f"{baseUrl}{page}"

    soup=custom_request(url)

    last_page=None
    try:
        last_page=int(soup.select(".paging a")[-1].text)
    except:
        last_page=1
    return last_page

# 1. get ids

In [5]:
last_page=get_last_page(date)

processed_date=date.replace("-","")
baseUrl=f"https://coinreaders.com/search.html?submit=submit&search=%EC%9D%B4%EB%8D%94%EB%A6%AC%EC%9B%80&search_exec=all&news_order=1&search_section=all&search_and=1&search_start_day={processed_date}&search_end_day={processed_date}&page="

full_ids=[]
for page in range(1,last_page+1):
    soup=custom_request(f"{baseUrl}{page}")
    ids = [a.get("href") for a in soup.select(".search_result_list_box > dl > dt > a")]
    full_ids.extend(ids)
    
file=open(f"ids/{date}.txt","w")
file.write(",".join(ids))
file.close()

# 2. get news

In [6]:
import pandas as pd

filename=f"ids/{date}.txt"
file=open(filename,"r")
ids=file.read().split(",")

baseUrl="https://www.coinreaders.com"

news_list=[]
for id in ids:
    url = f"{baseUrl}{id}"

    # 브라우저 유사 헤더
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/123.0.0.0 Safari/537.36"
        ),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Encoding": "gzip, deflate, br",
        # 목록 페이지 → 상세 페이지 흐름을 재현하려면 Referer를 실제 이전 페이지로 설정
        "Referer": baseUrl + id,
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
    }

    # 세션 + 재시도(403/429/5xx 백오프)
    retry = Retry(
        total=5,
        backoff_factor=1,  # 1,2,4,8,...
        status_forcelist=[403, 429, 500, 502, 503, 504],
        allowed_methods=["GET", "HEAD", "OPTIONS"]
    )
    adapter = HTTPAdapter(max_retries=retry)

    with requests.Session() as s:
        s.headers.update(headers)
        s.mount("https://", adapter)
        s.mount("http://", adapter)

        # 너무 빠른 요청을 피하기 위한 랜덤 지연
        time.sleep(random.uniform(0.8, 1.6))

        res = s.get(url, timeout=15)
        if res.status_code!=200:
            print(f"crawling failed from page {page}!")
            break
    
    soup=BeautifulSoup(res.text, 'html.parser')
    temp="".join([element.text for element in soup.select("#textinput > p") if element.text!='\xa0'])
    if temp=='':
        temp=soup.select_one("#textinput").text
    news_list.append(temp)
    
df = pd.DataFrame(
    [{"news": n, "date": date.replace("-", ".")} for n in news_list],
    columns=["news", "date"],
)
df.to_csv(f"news_list/{date}.csv", index=False)

# 3. llm 감성 분석

In [7]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv(verbose=True)

# 1) 출력 스키마 정의
class Sentiment(BaseModel):
    label: Literal["negative", "neutral", "positive"] = Field(..., description="감성 라벨")

parser = JsonOutputParser(pydantic_object=Sentiment)

# 2) 프롬프트 템플릿
system_text = """
### 작업 정의
당신은 암호화폐 시장 전문 감성 분석가입니다. 주어진 한국어 암호화폐 뉴스가 해당 코인(비트코인, 이더리움 등)의 가격에 미칠 영향을 분석하여 긍정(positive), 부정(negative), 중립(neutral) 중 하나로 분류하세요.

### 분류 기준
- **긍정(positive)**: 가격 상승, 기술 발전, 규제 완화, 기관 투자 증가, 채택 확대 등 호재성 내용
- **부정(negative)**: 가격 하락, 보안 사고, 규제 강화, 시장 불안, 청산 사태 등 악재성 내용
- **중립(neutral)**: 단순 사실 전달, 양면적 내용, 명확한 방향성 없는 시장 분석

### Few-Shot 예시

**예시 1:**
뉴스: "이더리움은 모멘텀 반등하며 최근 몇 달 동안 MTM 지표가 상승세를 나타내고 단기 흐름과 장기 추세가 점차 일치하고 있다."
result: positive

**예시 2:**
뉴스: "비트코인 이더리움 시세가 모두 단기적으로 내림세를 보일 가능성이 크다는 조사기관의 관측이 나왔다."
result: negative

**예시 3:**
뉴스: "디지털 자산은 증권 토큰, 상품 토큰, 상업 및 소비자용 토큰의 세 가지 범주로 나눌 것을 제안한다."
result: neutral

**예시 4:**
뉴스: "트럼프를 필두로 한 미국 행정부가 유니스왑 등 이더리움 기반 탈중앙화 금융을 직접 언급하면서 제도권 편입이 필요하다고 지적했다."
result: positive

**예시 5:**
뉴스: "가상화폐 시장 역사상 24시간 기준 최대 청산 사태가 발생했으며 이더리움은 최고점 대비 12.2% 급락했다."
result: negative

**예시 6:**
뉴스: "암호화폐 거래소가 자회사의 가상자산사업자(VASP) 권한을 활용해 거래소 출범을 추진한다."
result: neutral

**예시 7:**
뉴스: "SEC가 내놓은 프로젝트 크립토는 미국 금융의 온체인화를 표방하며 이더리움에 긍정적인 영향을 미친다."
result: positive

**예시 8:**
뉴스: "비트코인은 발표 직후 단 몇 초 만에 1만 달러 이상 급락했고 이더리움과 주요 알트코인은 평균 60~70% 낙폭을 기록했다."
result: negative

**예시 9:**
뉴스: "CFTC는 비트코인과 이더를 상품으로, SEC는 특정 디지털 자산을 증권으로 간주하는 분류 체계를 수립한다."
result: neutral

### 주의사항
1. **한국 시장 특성 반영**: 김치 프리미엄, 국내 거래소 이슈, 한국 규제 동향 고려
2. **숫자와 맥락 연계**: 단순 가격 변동률보다 전후 맥락과 시장 흐름 분석
3. **혼합 감성 처리**: 긍정/부정 요소가 혼재된 경우 전체적 논조의 우세한 방향 선택
4. **시간 민감성**: "급락 후 급반등" 같은 단기 변동성은 최종 상태에 초점
5. **규제/기술 뉴스**: 장기적 영향력을 평가 (예: DeFi 제도권 편입은 긍정)

### 출력 형식
result: [positive/negative/neutral]
"""

human_text = """
본문: {body}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_text),
        ("human", human_text),
    ]
).partial(format_instructions=parser.get_format_instructions())

# 3) 모델 정의 (환경변수 OPENAI_API_KEY 필요)
OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=OPENAI_API_KEY)  # 임의 모델. 교체 가능

# 4) 체인 구성: Prompt -> LLM -> JSON 파서
chain = prompt | llm | parser

/Users/dogiheon/Desktop/invigoworks/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
def parseResponse(label):
    if label=="negative":
        return -1
    elif label=="neutral":
        return 0
    else:
        return 1

In [9]:
df=pd.read_csv(f"news_list/{date}.csv")

In [10]:
news_list=df["news"].tolist()

labels=[0]*len(news_list)

In [11]:
for idx, news in enumerate(news_list):
    labels[idx]=parseResponse(chain.invoke(news)["label"])
    print(f"{idx+1}/{len(news_list)} finished...")

1/5 finished...
2/5 finished...
3/5 finished...
4/5 finished...
5/5 finished...


In [12]:
df["label"]=labels
df.to_csv(f"news_list_group_by_date/{date.replace("-",".")}.csv", index=False)

In [13]:
print(f"{date} finished")

2025-11-11 finished
